In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
import os

cloned_repo_path = os.path.abspath('')#insert path here
sys.path.insert(0, cloned_repo_path)
cloned_repo_path = os.path.abspath('.')
sys.path.insert(0, cloned_repo_path)

In [ ]:
import os
import pickle 
import stumpy
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

## Helping Functions

In [ ]:
def plot_all_dimensions(df, filepath, sensor_id_type_mapping):
    fig, axs = plt.subplots(df.shape[1], sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, df.shape[1] * 5))
    for k, dim_name in enumerate(df.columns):
        axs[k].set_ylabel(dim_name, fontsize=10)
        axs[k].set_xlabel('Time', fontsize=10) 
        axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])
        axs[k].legend(loc="upper right")
                
    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()


In [ ]:
def create_directory(directory="Results"):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print("Directory Created Successfully")
    
    else:
        print("Directory Already Exists")

In [ ]:
def calculate_and_visualize_mdls(final_data, mps, indices, filepath, m=2016):
    motifs_idx = np.argmin(mps, axis=1)

    # Getting the nearest neighbours of each individual motifs_idx

    nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]

    mdls, subspaces = stumpy.mdl(final_data, m, motifs_idx, nn_idx)

    print(f"MDLS:\n\t {mdls}\n")

    print(f"Subspaces:\n\t {subspaces}\n")

    k = np.argmin(mdls)

    print(f"Suugested Columns For Multidimensional Matrix Profile:\n\t {final_data.columns[subspaces[k]]}\n")
    
    plt.plot(np.arange(len(mdls)), mdls, c='red', linewidth='4')
    plt.xlabel('k (zero-based)', fontsize='20')
    plt.ylabel('Bit Size', fontsize='20')
    plt.xticks(range(mps.shape[0]))
    
    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()
    
    return final_data.columns[subspaces[k]].tolist(), final_data.columns[subspaces[-1]].tolist()

In [ ]:
def mps_calculations_mstump_m(final_data, filepath, m=2016):
    # Calculating Multidimensional Matrix Profile For 

    mps, indices = stumpy.mstump_m(final_data, m)

    # Displaying the shape of MultiDimensional Matrix Profile 

    print(f"MPS Shape: {mps.shape}")

    # Getting the motifs_idx based on the standard np.argmin method 
    # It gives 1 single motif for each dimension

    motifs_idx = np.argmin(mps, axis=1)

    # Displaying the motifs of each dimension
    
    print(f"Motif Start Index: {motifs_idx}")

    nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]
    
   
    print(f"Nearest Start Index: {nn_idx}")


    df = final_data.reset_index(drop=True)

    # Plotting the dimensions of the time series dataframe df a
  

    fig, axs = plt.subplots(mps.shape[0] * 2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, mps.shape[0] * 10))
    label = ''
    for k, dim_name in enumerate(df.columns):
        axs[k].set_ylabel(dim_name, fontsize=10)
        axs[k].set_xlabel('Time', fontsize=10) 
        axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])
        axs[k].legend(loc="upper right")
        axs[k].plot(range(motifs_idx[k], motifs_idx[k] + m), df[dim_name].iloc[motifs_idx[k] : motifs_idx[k] + m], c='red', linewidth=4)
        axs[k].plot(range(nn_idx[k], nn_idx[k] + m), df[dim_name].iloc[nn_idx[k] : nn_idx[k] + m], c='red', linewidth=4)

        label = label + ', ' + dim_name
        
        axs[k].axvline(x=motifs_idx[k], linestyle="dashed", c='black')
        axs[k].axvline(x=nn_idx[k], linestyle="dashed", c='black')

        axs[k + mps.shape[0]].set_ylabel(f"P_{k}", fontsize=10)
        axs[k + mps.shape[0]].plot(mps[k], c='orange', label=f"{label.strip(',')}")
        axs[k + mps.shape[0]].set_xlabel('Time', fontsize=10)    

        axs[k + mps.shape[0]].axvline(x=motifs_idx[k], linestyle="dashed", c='black')
        axs[k + mps.shape[0]].axvline(x=nn_idx[k], linestyle="dashed", c='black')    

        axs[k + mps.shape[0]].plot(motifs_idx[k], mps[k, motifs_idx[k]] + 1, marker="v", markersize=10, color='red')
        axs[k + mps.shape[0]].plot(nn_idx[k], mps[k, nn_idx[k]] + 1, marker="v", markersize=10, color='red')
        
        axs[k + mps.shape[0]].text(motifs_idx[k], mps[k, motifs_idx[k]], f"{1}m", fontsize="xx-large")
        axs[k + mps.shape[0]].text(nn_idx[k], mps[k, motifs_idx[k]], f"{1}n", fontsize="xx-large")
        axs[k + mps.shape[0]].legend(loc="upper right")
        
    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()
    
    return mps, indices



In [ ]:
def mps_calculations_mstump(final_data, filepath, m=2016):
    # Calculating Multidimensional Matrix Profile For a weekly rhythm

    mps, indices = stumpy.mstump(final_data, m)
    print(f"MPS Shape: {mps.shape}")

    # Getting the motifs_idx based on the standard np.argmin method 
    motifs_idx = np.argmin(mps, axis=1)

    # Displaying the motifs of each dimension
      print(f"Motif Start Index: {motifs_idx}")


    nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]
    
    # Displaying the Nearest Neighbors of each dimension
    
    print(f"Nearest Neighbor Start Index: {nn_idx}")


    df = final_data.reset_index(drop=True)

    fig, axs = plt.subplots(mps.shape[0] * 2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, mps.shape[0] * 10))
    label = ''
    for k, dim_name in enumerate(df.columns):
        axs[k].set_ylabel(dim_name, fontsize=10)
        axs[k].set_xlabel('Time', fontsize=10) 
        axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])
        axs[k].legend(loc="upper right")
        axs[k].plot(range(motifs_idx[k], motifs_idx[k] + m), df[dim_name].iloc[motifs_idx[k] : motifs_idx[k] + m], c='red', linewidth=4)
        axs[k].plot(range(nn_idx[k], nn_idx[k] + m), df[dim_name].iloc[nn_idx[k] : nn_idx[k] + m], c='red', linewidth=4)
        
        label = label + ', ' + dim_name
        
        axs[k].axvline(x=motifs_idx[k], linestyle="dashed", c='black')
        axs[k].axvline(x=nn_idx[k], linestyle="dashed", c='black')

        axs[k + mps.shape[0]].set_ylabel(f"P_{k}", fontsize=10)
        axs[k + mps.shape[0]].plot(mps[k], c='orange', label=f"{label.strip(',')}")
        axs[k + mps.shape[0]].set_xlabel('Time', fontsize=10)    

        axs[k + mps.shape[0]].axvline(x=motifs_idx[k], linestyle="dashed", c='black')
        axs[k + mps.shape[0]].axvline(x=nn_idx[k], linestyle="dashed", c='black')    

        axs[k + mps.shape[0]].plot(motifs_idx[k], mps[k, motifs_idx[k]] + 1, marker="v", markersize=10, color='red')
        axs[k + mps.shape[0]].plot(nn_idx[k], mps[k, nn_idx[k]] + 1, marker="v", markersize=10, color='red')
        
        axs[k + mps.shape[0]].text(motifs_idx[k], mps[k, motifs_idx[k]], f"{1}m", fontsize="xx-large")
        axs[k + mps.shape[0]].text(nn_idx[k], mps[k, motifs_idx[k]], f"{1}n", fontsize="xx-large")
        axs[k + mps.shape[0]].legend(loc="upper right")
        
    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()
    
    return mps, indices

In [ ]:
# Further Processing the motifs to get the first index of each corresponding motifs sequence

def get_percent_motif_start_index(motifs):
    final_motifs = []
    
    for i in range(0, len(motifs), 2016):
        final_motifs.append(motifs.index[i])
            
    return final_motifs

In [ ]:
# Further Processing the motifs to get the first index of each corresponding motifs sequence
# 

def get_motif_start_index(motifs):
    final_motifs = []
    j = 0
    check = False
    
    for i in range(0, len(motifs), 2016):
        if not check:
            j = i

        final_motifs.append(motifs[j])

        if check:
            j = i + value

        elif motifs[i] + 2016 > len(motifs):
            value = len(motifs) - motifs[i]
            j = i + value
            check = True
            
                
    return final_motifs


In [ ]:
# A function that would take in percentage

def select_motifs_discords_percentage(mps, dimension, motif_percentage, discord_percentage, motifs={}, discords={}):
    motif_threshold = mps.quantile(motif_percentage/100)
    discord_threshold = mps.quantile((100 - discord_percentage)/100)
    
    motif = mps[mps < motif_threshold]
    discord = mps[mps > discord_threshold]
    
    if len(motif):
        
        motifs[dimension] = get_percent_motif_start_index(motif)
        
    return motifs, discords

In [ ]:
# Select the Upper K of all points and Lowest J 
def select_top_k_motifs_discords(mps, dimension, k_motifs, k_discords, motifs={}, discords={}):

    sorted_mps = np.argsort(mps, kind='stable')
    
    motifs[dimension] = get_motif_start_index(sorted_mps)[:k_motifs]
    
    return motifs, discords

In [ ]:
# Taking user's input for function 

motif_thresholds_for_all_dimensions = {}

def take_function_and_function_parameters_input():
    motif_thresholds_for_all_dimensions = {}
    for dimension in range(mps.shape[0]):
        motif_thresholds_for_single_dimension = {}
        function = input(f"Select A Function To Be Applied to Dimension {dimension}\n"
                         f"\tPress 1 For Selection Based on Percentage\n "
                         f"\tPress 2 For Top K Motifs and Discords Selection: ")

        if int(function) == 1:
            motif_percentage = input(f"\tEnter a Specific Threshold Value For Motif Selection For Dimension {dimension}: ")
            discord_percentage = input(f"\tEnter a Specific Threshold Value For Discord Selection For Dimension {dimension}: ")
            motif_thresholds_for_single_dimension["function"] = int(function)
            motif_thresholds_for_single_dimension["motif_percentage"] = int(motif_percentage)
            motif_thresholds_for_single_dimension["discord_percentage"] = int(discord_percentage)

        elif int(function) == 2:
            k_motif = input(f"\tEnter Top K Motif Selection For Dimension {dimension}: ")
            k_discord = input(f"\tEnter Top K Discord Selection For Dimension {dimension}: ")
            motif_thresholds_for_single_dimension["function"] = int(function)
            motif_thresholds_for_single_dimension["k_motifs"] = int(k_motif)
            motif_thresholds_for_single_dimension["k_discords"] = int(k_discord)
        else:
            continue
        motif_thresholds_for_all_dimensions[dimension] = motif_thresholds_for_single_dimension
        
    return motif_thresholds_for_all_dimensions


In [ ]:
# Calculating Motifs and Discords 


def calculate_motifs_discords_for_each_dimension(mps, motif_thresholds_for_all_dimensions):
    motifs = {}
    discords = {}

    mps_df = pd.DataFrame(mps).T

    for key, value in motif_thresholds_for_all_dimensions.items():
        if value.get('function') == 1:
            motifs, discords = select_motifs_discords_percentage(mps_df[key], key, value["motif_percentage"],
                                                                 value["discord_percentage"], motifs, discords)
        elif value.get('function') == 2:
            motifs, discords = select_top_k_motifs_discords(mps_df[key], key, value['k_motifs'], 
                                                            value['k_discords'], motifs, discords)
    return motifs, discords

In [ ]:
# Selecting Nearest Neighbour 

def calculate_nn_and_filter_motifs(motifs):
    final_nn = {}
    final_motifs = {}

    for key, values in motifs.items():
        nns = {}
        for value in values:
            if nns.get(value) is None:
                if final_nn.get(key):
                    final_nn[key].append(indices[key, value])
                    final_motifs[key].append(value)
                else:
                    final_nn.setdefault(key, []).append(indices[key, value])
                    final_motifs.setdefault(key, []).append(value)

                nns[indices[key, value]] = 1
    
    return final_motifs, final_nn

In [ ]:

def create_sensor_id_type_mapping(location_input):
    sensors_id_type = sensor_location_type_ids.loc[:,location_input].to_dict()

    sensor_id_type_mapping = {}
    for key, values in sensors_id_type.items():
        if isinstance(values, str)  and 'list' not in key:
            for value in eval(values.replace(' ', ',')):
                sensor_id_type_mapping[f"{value}"] = key
            
    return sensor_id_type_mapping

In [ ]:
# Plotting the dimensions of the time series dataframe 

def plot_results_of_mps(df, mps, m, final_motifs, final_nn, sensor_id_type_mapping, filepath):
    fig, axs = plt.subplots(mps.shape[0] * 2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(25, mps.shape[0] * 10))
    label = ''
    for k, dim_name in enumerate(df.columns):
        axs[k].set_ylabel(dim_name, fontsize=10)
        axs[k].set_xlabel('Time', fontsize=10) 
        axs[k].plot(df[dim_name], label=sensor_id_type_mapping[dim_name])
        axs[k].legend(loc="upper right")
        i = 0
        if final_motifs.get(k) and final_nn.get(k):
            for motifs_idx, nn_idx in zip(final_motifs.get(k), final_nn.get(k)):
                
                axs[k].plot(df[dim_name].iloc[motifs_idx : motifs_idx + m], c='red', linewidth=4)
                axs[k].plot(df[dim_name].iloc[nn_idx : nn_idx + m], c='red', linewidth=4)
                axs[k].axvline(x=motifs_idx, linestyle="dashed", c='black')
                axs[k].axvline(x=nn_idx, linestyle="dashed", c='black')

                axs[k + mps.shape[0]].plot(motifs_idx, mps[k, motifs_idx] + 1, marker="v", markersize=10, color='red')
                axs[k + mps.shape[0]].plot(nn_idx, mps[k, nn_idx] + 1, marker="v", markersize=10, color='red')

                axs[k + mps.shape[0]].axvline(x=motifs_idx, linestyle="dashed", c='black')
                axs[k + mps.shape[0]].axvline(x=nn_idx, linestyle="dashed", c='black')

                axs[k + mps.shape[0]].text(motifs_idx, mps[k][motifs_idx], f"{i+1}m", fontsize="xx-large")
                axs[k + mps.shape[0]].text(nn_idx, mps[k][nn_idx], f"{i+1}n", fontsize="xx-large")

                i += 1
        label = label + ', ' + dim_name
        
        axs[k + mps.shape[0]].set_ylabel(f"P_{k}", fontsize=10)
        axs[k + mps.shape[0]].plot(mps[k], c='orange', label=f"{label.strip(',')}")
        axs[k + mps.shape[0]].set_xlabel('Time', fontsize=10)
        axs[k + mps.shape[0]].legend(loc="upper right")

    plt.savefig(filepath, transparent=False, bbox_inches='tight')
    plt.show()


In [ ]:
def apply_functions(final_data, mps, m, filepath):
    motif_thresholds_for_all_dimensions = take_function_and_function_parameters_input()

    # Displaying Dictionary of Each Dim

    print(f"\nMotif Threshold Dictionary:\n\t {motif_thresholds_for_all_dimensions}\n")

    motifs, discords = calculate_motifs_discords_for_each_dimension(mps, motif_thresholds_for_all_dimensions)

    print(f"Motifs Before Filtering:\n\t {motifs}\n")

    final_motifs, final_nn = calculate_nn_and_filter_motifs(motifs)

    df = final_data.reset_index(drop=True)

    sensor_id_type_mapping = create_sensor_id_type_mapping(location_input)

    plot_results_of_mps(df, mps, m, final_motifs, final_nn, sensor_id_type_mapping, filepath)

## Creating Folders and Reading Preprocessed Data Files

In [ ]:
create_directory(directory="Results")

In [ ]:
sensor_location_type_ids = pd.read_csv('./Processed_Data/Sensor_Location_Type_Ids.csv', index_col='name')

In [ ]:
with open('./Processed_Data/sensor_type_names_dict.pkl', 'rb') as f:
    sensor_type_names_dict = pickle.load(f)

In [ ]:
location_input = input("Enter a Specific Location For MMP: ")

In [ ]:
directory = "Results/" + location_input.replace('/', '_')
create_directory(directory)

In [ ]:
directory = directory + '_User_Interaction'
create_directory(directory)

In [ ]:
columns = []
bool_columns = []
decimal_columns = []

for key in sensor_type_names_dict.keys():
    if key == 'bool':
        bool_columns.extend(eval(sensor_location_type_ids.loc[f'{key}_list', location_input]))
    elif key == 'decimal':
        decimal_columns.extend(eval(sensor_location_type_ids.loc[f'{key}_list', location_input]))   
    columns.extend(eval(sensor_location_type_ids.loc[f'{key}_list', location_input]))
    
columns.append('Timestamp')

In [ ]:
# Reading Time Series DataFrame

final_data = pd.read_csv("./Processed_Data/Final_Sensor_Time_Series_Imputed.csv", usecols=columns, 
                         index_col='Timestamp', parse_dates=True)


In [ ]:
final_data

In [ ]:

sensors_id_type = sensor_location_type_ids.loc[:,location_input].to_dict()

sensor_id_type_mapping = {}
for key, values in sensors_id_type.items():
    if isinstance(values, str)  and 'list' not in key:
        for value in eval(values.replace(' ', ',')):
            sensor_id_type_mapping[f"{value}"] = key

In [ ]:
sensor_id_type_mapping

In [ ]:
plot_all_dimensions(final_data, f"{directory}/All_Dimensions.jpg", sensor_id_type_mapping)

## Selecting Only the Relevant Columns
    Filter out the columns which do not look good and might ruin the results.  

In [ ]:
select_choice = input("Do You Want Filter Specific Columns to be used for MMP Calculation"
                      "\n\t Press 1 to Select the Columns"
                      "\n\t Press Any Other Key to Use All the Columns (No Filtering):\n")

new_final_data = final_data.copy()

if select_choice == '1':
    select_dimensions = input("Enter Comma Separated Sensor Ids to be used for MMP Calculation: ")
    columns_to_use = select_dimensions.split(',')
    columns_to_use = [value.strip() for value in columns_to_use]
    
    new_final_data = final_data[columns_to_use]
    plot_all_dimensions(new_final_data, f"{directory}/After_Filtering_Dimensions.jpg", sensor_id_type_mapping)

## Select The Columns Which Need to be Aggregated Using Mean
    You can select columns by device type to be aggregated in 1 selection like All CO2 columns need to be aggregated than add a comma separated sensor ids for all CO2 columns as I entered 2764,2777,2760 in 1 selection and when prompted again I selected all Humididty sensors 2763,2776,2759

In [ ]:
select_choice_mean = input("Do You Want to Calculate Mean over Specific Selection of Columns"
                      "\n\t Press 1 to Select the Columns to Calculate Mean"
                      "\n\t Press Any Other Key For No Mean:\n")

mean_column_pairs = {}
i = 1

while select_choice_mean == '1':
    select_dimensions_mean = input("Enter Comma Separated Sensor Ids to Calculate Mean: ")
    
    columns_for_mean = select_dimensions_mean.split(',')
    columns_for_mean = [value.strip() for value in columns_for_mean]
    
    mean_column_pairs[f'Mean_{i}'] = columns_for_mean
    sensor_id_type_mapping[f'Mean_{i}'] = select_dimensions_mean
    
    select_choice_mean = input("Do You Want to Calculate Mean for Some Other Selection of Columns"
                      "\n\t Press 1 to Select the Columns to Calculate Mean"
                      "\n\t Press Any Other Key For No More Mean Calculation:\n")
    i += 1
    
    


In [ ]:
if mean_column_pairs:
    mean_columns = []

    for key, value in mean_column_pairs.items():
        new_final_data[key] = final_data[value].mean(axis=1)
        mean_columns.extend(value)

    new_final_data = new_final_data[list(set(new_final_data.columns) - set(mean_columns))]
    plot_all_dimensions(new_final_data, f"{directory}/After_Applying_Mean_to_Dimensions.jpg", sensor_id_type_mapping)

## Select The Columns Which Need to be Aggregated Using Sum
    You can select columns by device type to be aggregated in 1 selection. I did not apply it in this example

In [ ]:
select_choice_sum = input("Do You Want to Calculate Sum over Specific Selection of Columns"
                      "\n\t Press 1 to Select the Columns to Calculate Sum"
                      "\n\t Press Any Other Key For No Sum:\n")

sum_column_pairs = {}
i = 1

while select_choice_sum == '1':
    select_dimensions_sum = input("Enter Comma Separated Sensor Ids to Calculate Sum: ")
    
    columns_for_sum = select_dimensions_sum.split(',')
    columns_for_sum = [value.strip() for value in columns_for_sum]
    
    sum_column_pairs[f'Sum_{i}'] = columns_for_sum
    sensor_id_type_mapping[f'Sum_{i}'] = select_dimensions_sum
    
    select_choice_sum = input("Do You Want to Calculate Sum for Some Other Selection of Columns"
                      "\n\t Press 1 to Select the Columns to Calculate Sum"
                      "\n\t Press Any Other Key For No More Sum Calculation:\n")
        
    i += 1
    


In [ ]:
if sum_column_pairs:
    sum_columns = []

    for key, value in sum_column_pairs.items():
        new_final_data[key] = final_data[value].sum(axis=1)
        sum_columns.extend(value)

    new_final_data = new_final_data[list(set(new_final_data.columns) - set(sum_columns))]
    plot_all_dimensions(new_final_data, f"{directory}/After_Applying_Sum_to_Dimensions.jpg", sensor_id_type_mapping)

### MPS Calculation Using Constraint Algorithm For All Bool Dimensions (Weekly Rhythm)

In [ ]:
mps, indices = mps_calculations_mstump_m(new_final_data, 
                                        f"{directory}/Selected_Dimensions_mstump_m_Weekly.jpg", 
                                        m=2016)

### Dimensionality Reduction Using MDLs for Selected Dimensions (Weekly Rhythm)

In [ ]:
select_choice_mdl = input("Do You Want to Apply MDL to the Data"
                      "\n\t Press 1 to Apply MDL"
                      "\n\t Press Any Other Key For No MDL:\n")

In [ ]:
if select_choice_mdl == '1':
    columns, reordered_columns = calculate_and_visualize_mdls(new_final_data, mps, indices, 
                                       f"{directory}/MDL_Ideal_Selected_Dimensions_Weekly.jpg",
                                       m=2016)

### MPS Calculation Using Constraint Algorithm For Reduced Selected Dimensions (Weekly Rhythm)

In [ ]:
select_choice_mdl_mmp = input("Do You Want to Calculate MMP Using Reduced Dimensions Suggested by MDL"
                      "\n\t Press 1 to Calculate MMP After MDL"
                      "\n\t Press Any Other Key For No MMP After MDL:\n")

In [ ]:
if select_choice_mdl_mmp == '1' and select_choice_mdl == '1':
    mps, indices = mps_calculations_mstump_m(new_final_data[columns], 
                                        f"{directory}/MDL_Reduced_Dimensions_mstump_m_Weekly.jpg", 
                                        m=2016)